In [2]:
import numpy as np
import pandas as pd 
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten
import torch as to
import torch.nn as nn 
import torch.nn.functional as func
from torch.utils import data
from torch.utils.data import DataLoader
import random 
import time 
import torch.nn.utils.rnn as rnnutils
from sklearn.model_selection import train_test_split
import torchviz
from torchsummary import summary
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score 
from itertools import tee
import hashlib

Using TensorFlow backend.



### Info 
This notebook is a continuation of the 'baseline.ipynb' nootebook. 

### Setup 
+> Added balancing of the data-set. Balancing is done in a way that samples are added until the less frequent class isn't same as more frequent class. Samples are added squentialy, not randomly, thus ensuring that the differenece of the most frequent sample and least frequent sample in the data-set is one.

+> f1 is now macro

+> batchnormalization was added befor convolution and removed :D 

+> we are currently stopping the training if f1 is not changing (max_constant_f1 denotes the max number of epochs that the f1 is not changing)

+> additonal data was added to the results dataframe (precision and recall for each class)

+> to ease the strain on memory this code has all input data is represented as indices, thus it runs a bit slower because it has to access the input dataframe to attain input vectors

+> original dataframe is preprocesed and deleted so that memory is freed 

+> now you can specify a gpu, ouput of all results, predictions and models (CONSTANT: 'MODELS_NAME')

In [3]:
#CELL METADATA
DATASET = '/home/mbosnjak/Datasets/embedded_comments_last_100_min_20com_per_auth_w_wc_10_200_no_mbti.csv'
FOLDS = '/home/mbosnjak/Datasets/comments_last_100_min_20com_per_auth_w_wc_10_200.csv.folds.csv'
AUTHORS = '/home/mbosnjak/Datasets/author_profiles.csv'

MODELS_NAME = 'baseline_take2'
OUTPUT_DIR = f'mpr_{MODELS_NAME}/'
RESULTS = OUTPUT_DIR + f'results_{MODELS_NAME}.csv'

RELEVANT_DIFFERENCE = 0.0000001
NUMBER_OF_CLASSES = 2
STATE = 156
no_of_folds = 2
cuda_device_index = 0

mbti_labels_1 = ['introverted', 'intuitive']
mbti_labels_2 = ['thinking', 'perceiving']
mbti_labels = [mbti_labels_1[0]] #+ mbti_labels_2
MODELS_PREFORMANCE_COLUMNS = ['val_f1', 'val_precision_0', 'val_precision_1', 'val_precision_macro' ,\
                              'val_recall_0', 'val_recall_1', 'val_recall_macro', \
                              'test_f1', 'test_precision_0', 'test_precision_1', 'test_precision_macro',\
                              'test_recall_0', 'test_recall_1', 'test_recall_macro',\
                              'epoch']
MODELS_PARAMETER_COLUMNS = ['mbti_trait', 'fold', 'learning_rate', 'batch_size',\
                           'kernels_count', 'sentences_count' ]
MODELS_META_DATA_COLUMNS = ['models_name', 'hash_id']
to.manual_seed(STATE)

In [4]:
#CELL PARAMETERS
batch_sizes = [16, 32, 64]
learning_rates = [0.00001, 0.000001, 0.0000005, 0.0000001, 0.00000005]
kernel_sizes = [8, 12, 16]
sentences_counts = [2, 4, 6]
n_epochs = 90
use_GPU = True
#TODO: this is sentences per author not comments per author, quick fix is max size 200*100 (200 max sentences by comment * 100 comments)
comments_per_author = 20000
#comments_per_author = 20
cuda_device = to.device(cuda_device_index) if use_GPU else None
validation_p = .2
print_status_batch = 1000
max_constant_f1 = 2

In [4]:
def get_checkpoints_name(models_identifier):
    return OUTPUT_DIR+"save_"+MODELS_NAME+'_'+models_identifier + ".pt"

def get_predictions_file_name(models_identifier, target, fold):
    return OUTPUT_DIR + f'{target}_{fold}_{MODELS_NAME}_{models_identifier}_predictions'

def delimiter():
    print('-'*23)

In [5]:
input_df = pd.read_csv(DATASET)
#input_df = pd.read_csv(DATASET, nrows = 2000) # TODO: remove nrows
folds_df = pd.read_csv(FOLDS, usecols =['author', 'fold'])
authors_profiles_df = pd.read_csv(AUTHORS)
authors_with_mbti = list(authors_profiles_df[authors_profiles_df.introverted.notnull()]['author'])

In [7]:
#TODO: add comment delimiter -> to recognize comments end and beginning
def merger(comments):
    npad = [(0, max(sentences_counts)), (0, 0)]
    carpet = comments[:comments_per_author].as_matrix(columns=comments.columns[1:1025])
    carpet_padded = np.pad(carpet, pad_width=npad, mode='constant', constant_values=0)
    #print(f'before {carpet.shape} - after {carpet_padded.shape}')
    return to.tensor(carpet_padded)

In [9]:
#Data preparation
#data_df = input_df[input_df['author'].isin(authors_with_mbti)]
#del input_df
#present_authors = sorted(data_df['author'].unique())
#authors_indices = dict(zip(present_authors , list(range(len(present_authors)))))
input_x_df = data_df.groupby(['author']).apply(merger).tolist()
del data_df

MemoryError: Unable to allocate array with shape (1024, 2202312) and data type float64

In [21]:
results = pd.DataFrame(data=None, columns=MODELS_META_DATA_COLUMNS+MODELS_PREFORMANCE_COLUMNS+MODELS_PARAMETER_COLUMNS)
for name in MODELS_PREFORMANCE_COLUMNS:
    results = results.astype({name: 'float64'})

In [ ]:
resutls = pd.read_csv('mpr_baseline_take2/results_baseline_take2.csv',  index_col=0)

In [22]:
def set_hash(index):
    global results
    identifier = hashlib.md5(''.join([str(x) for x in results[MODELS_PARAMETER_COLUMNS].iloc[index]]).encode('utf-8')).hexdigest()
    results.at[index, 'hash_id'] = identifier
    return identifier

def create_new_model_input(mbti_trait, fold, learning_rate, batch_size,\
                           kernels_count, sentences_count):
    global results
    results = results.append({'models_name':MODELS_NAME, 'mbti_trait' : mbti_trait, 'fold' : fold, 'learning_rate' : learning_rate, 'batch_size' : batch_size, 'kernels_count' : kernels_count, 'sentences_count' : sentences_count}, ignore_index=True)
    return set_hash(len(results)-1)    

def update_models_val_results(models_identifier, val_f1, val_precision_0, val_precision_1, val_precision_m, val_recall_0, val_recall_1, val_recall_m, epoch):
    global results
    results.loc[results['hash_id'] == models_identifier, 'val_f1'] = val_f1
    results.loc[results['hash_id'] == models_identifier, 'val_precision_0'] = val_precision_0
    results.loc[results['hash_id'] == models_identifier, 'val_precision_1'] = val_precision_1
    results.loc[results['hash_id'] == models_identifier, 'val_precision_macro'] = val_precision_m
    results.loc[results['hash_id'] == models_identifier, 'val_recall_0'] = val_recall_0
    results.loc[results['hash_id'] == models_identifier, 'val_recall_1'] = val_recall_1
    results.loc[results['hash_id'] == models_identifier, 'val_recall_macro'] = val_recall_m
    results.loc[results['hash_id'] == models_identifier, 'epoch'] = epoch
    
def update_models_test_results(models_identifier, test_f1, test_precision_0, test_precision_1, precision_m, test_recall_0, test_recall_1, test_recall_m):
    global results
    results.loc[results['hash_id'] == models_identifier, 'test_f1'] = test_f1
    results.loc[results['hash_id'] == models_identifier, 'test_precision_0'] = test_precision_0
    results.loc[results['hash_id'] == models_identifier, 'test_precision_1'] = test_precision_1
    results.loc[results['hash_id'] == models_identifier, 'test_precision_macro'] = test_precision_m
    results.loc[results['hash_id'] == models_identifier, 'test_recall_0'] = test_recall_0
    results.loc[results['hash_id'] == models_identifier, 'test_recall_1'] = test_recall_1
    results.loc[results['hash_id'] == models_identifier, 'test_recall_macro'] = test_recall_m
    
def get_best_models_data(target, fold):
    return results.loc[results[(results.mbti_trait == target) & (results.fold == fold)]['val_f1'].idxmax()]
    

In [23]:
create_new_model_input('dfsfdsf',333,3,223333,1,13)

'e5a677ccc00e12a5dc84842655a04262'

In [35]:
results

,models_name,hash_id,val_f1,val_precision_0,val_precision_1,val_precision_macro,val_recall_0,val_recall_1,val_recall_macro,test_f1,...,test_recall_0,test_recall_1,test_recall_macro,epoch,mbti_trait,fold,learning_rate,batch_size,kernels_count,sentences_count
0,baseline_take2,e5a677ccc00e12a5dc84842655a04262,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,dfsfdsf,333,3,223333,1,13
1,baseline_take2,e36ae0804472af283afbd748b7cc30e9,0.440665,0.000000,0.787836,0.393918,0.0,0.787836,0.5,NaN,...,NaN,NaN,NaN,0.0,introverted,0,0.001,1,4,2
2,baseline_take2,2f4e282eb0a31414afec1306d8c24742,0.175029,0.212164,0.000000,0.106082,1.0,0.000000,0.5,NaN,...,NaN,NaN,NaN,0.0,introverted,0,0.01,1,4,2
3,baseline_take2,eb66caa5ee9eb650d51577d0a28f7fe2,0.440665,0.000000,0.787836,0.393918,0.0,0.787836,0.5,NaN,...,NaN,NaN,NaN,0.0,introverted,0,0.0001,1,4,2
4,baseline_take2,2bb3311afd88c876e4c191400a913072,0.440665,0.000000,0.787836,0.393918,0.0,0.787836,0.5,NaN,...,NaN,NaN,NaN,0.0,introverted,0,1e-05,1,4,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
141,baseline_take2,5b27ff53feb4f2bf11f666f6dc4e45cc,0.440665,0.000000,0.787836,0.393918,0.0,0.787836,0.5,NaN,...,NaN,NaN,NaN,0.0,introverted,0,1e-07,4,16,4
142,baseline_take2,b7007e14266db7c6997a0309c265bb70,0.440665,0.000000,0.787836,0.393918,0.0,0.787836,0.5,NaN,...,NaN,NaN,NaN,0.0,introverted,0,1e-08,4,16,4
143,baseline_take2,e4504e69c4c25b2dcf922902c1560d0e,0.440665,0.000000,0.787836,0.393918,0.0,0.787836,0.5,NaN,...,NaN,NaN,NaN,0.0,introverted,0,1e-09,4,16,4
144,baseline_take2,e95b2e301c1a3c1985001f0379bbc17c,0.175029,0.212164,0.000000,0.106082,1.0,0.000000,0.5,NaN,...,NaN,NaN,NaN,0.0,introverted,0,1e-10,4,16,4


### FLOW

In [24]:
def get_input_output(data_authors, target):
    output_prep = authors_profiles_df[authors_profiles_df['author'].isin(data_authors)][target].tolist()
    input_indices = [index for author, index in authors_indices.items() if author in data_authors]
    return input_indices, output_prep

def balance_binary_data(input_df, output_df):
    positive_indices = [index for index, element in enumerate(output_df) if element == 1.0]
    negative_indices = [index for index, element in enumerate(output_df) if element == 0.0]
    negative_count = len(negative_indices)
    positive_count = len(positive_indices)
    
    if(positive_count > negative_count):
        return create_balanced_data(positive_indices, negative_indices, input_df, output_df)
    elif(negative_count > positive_count):
        return create_balanced_data(negative_indices, positive_indices, input_df, output_df)
        
def create_balanced_data(more_frequent, less_frequent, input_df, output_df):
    more_frequent_count = len(more_frequent)
    less_frequent_count = len(less_frequent)
    constant_multiplyer = more_frequent_count // less_frequent_count
    remaining_additions = more_frequent_count % less_frequent_count
    balanced_indices = less_frequent*constant_multiplyer + less_frequent[:remaining_additions] + more_frequent
    output_df_balanced = [output_df[index] for index in balanced_indices]
    input_df_balanced = [input_df[index] for index in balanced_indices]
    return input_df_balanced, output_df_balanced
    
def data_preparation(target, fold):
    test_data_authors = folds_df[(folds_df['fold'] == fold) & (folds_df['author'].isin(present_authors))]['author'].tolist()
    train_data_authors = folds_df[(folds_df['fold'] != fold) & (folds_df['author'].isin(present_authors))]['author'].tolist()
    train_input_indices, train_output = get_input_output(train_data_authors, target)
    test_input_indices, test_output = get_input_output(test_data_authors, target)

    train_input_indices, val_input_indices, train_output, val_output = train_test_split(train_input_indices, train_output, test_size=validation_p, random_state=STATE)
    assert len(train_input_indices) == len(train_output)
    assert len(test_input_indices) == len(test_output)
    assert len(val_input_indices) == len(val_output)
    train_input_indices, train_output = balance_binary_data(train_input_indices, train_output)
    return train_input_indices, train_output, val_input_indices, val_output, test_input_indices, test_output

def chunks(l, n):
    return [l[i:i + n] for i in range(0, len(l), n)]  

def create_minibatches(data_X, data_y, minibatch_size, cuda_dev):
    for idx_list in chunks(range(len(data_X)), minibatch_size):
        data_X_indices = [data_X[index] for index in idx_list]
        data_y_idx = [int(data_y[index]) for index in idx_list]
        
        minibatch_X = [input_x_df[index] for index in data_X_indices]
        minibatch_X = rnnutils.pad_sequence(minibatch_X, batch_first=True, padding_value = 0) 
        minibatch_X = minibatch_X.unsqueeze(1)
        minibatch_y = to.tensor(data_y_idx)
        if cuda_dev is not None:
            minibatch_X = minibatch_X#.to(device=cuda_dev, dtype=to.float)
            minibatch_y = minibatch_y.to(device=cuda_dev, dtype=to.long)

        yield((minibatch_X, minibatch_y))   


In [25]:
class ConvoCarpet(nn.Module):
    def __init__ (self, embedding_size = 1024, kernels_count=64, sentences_count=2):
        super(ConvoCarpet, self).__init__()
        self.batch_norm = nn.BatchNorm2d(1)
        self.conv_layer = nn.Conv2d(1, kernels_count, [sentences_count, embedding_size])
        self.pool_layer = nn.AdaptiveMaxPool2d((1, None))
        self.fc_layer = nn.Linear(kernels_count, NUMBER_OF_CLASSES)
        self.softmax = nn.Softmax(dim=1)
        
    def forward(self, input_batch):
        #normalized_input = self.batch_norm(input_batch)
        conv_output = func.relu(self.conv_layer(input_batch))
        maxpool_output = self.pool_layer(conv_output)
        maxpool_output = maxpool_output.flatten(start_dim=1)
        linear_output = self.fc_layer(maxpool_output)
        output = self.softmax(linear_output)
        #return func.log_softmax(linear_output)
        return output


In [26]:
def calcMeasures(total_logits, total_true, threshold):
    total_preds = []
    for minibatch_logits in total_logits:
        minibatch_preds = minibatch_logits[:,1] > threshold
        total_preds.append(minibatch_preds)
    total_preds = to.cat(total_preds)
    total_true = to.cat(total_true)
    F1 = f1_score(total_true.cpu(), total_preds.cpu(), pos_label = 1, average='macro')
    precision_m = precision_score(total_true.cpu(), total_preds.cpu(), average='macro')
    precision_1 = precision_score(total_true.cpu(), total_preds.cpu(), pos_label = 1)
    precision_0 = precision_score(total_true.cpu(), total_preds.cpu(), pos_label = 0)
    recall_m = recall_score(total_true.cpu(), total_preds.cpu(), average='macro')
    recall_1 = recall_score(total_true.cpu(), total_preds.cpu(), pos_label = 1)
    recall_0 = recall_score(total_true.cpu(), total_preds.cpu(), pos_label = 0)
    return F1, precision_0, precision_1, precision_m, recall_0, recall_1, recall_m

def apply_model(model, loss_func, data_loader):
    model.eval()
    total_loss = 0
    predicted = []
    true_output = []
    minibatch_loss = 0
    with to.no_grad():
        n_batch = 0
        for inputs, labels in data_loader:
            n_batch += 1
            minibatch_logits = model(inputs.to(device=cuda_device, dtype=to.float))
            minibatch_loss = loss_func(minibatch_logits, labels)
            
            total_loss += minibatch_loss
            predicted.append(minibatch_logits)
            true_output.append(labels)
            
    return total_loss, predicted, true_output

def optimize_models_params(trait, fold, train_input_indices, train_output, val_input_indices, val_output):
    print('Start training ...')
    
    loss = nn.CrossEntropyLoss()
    
    for batch_size in batch_sizes:
        for kernel_size in kernel_sizes:
            for sent_size in sentences_counts:
                for learning_rate in learning_rates:                
                    
                    train_loader = create_minibatches(train_input_indices, train_output, batch_size, cuda_device)
                    val_loader = create_minibatches(val_input_indices, val_output, batch_size, cuda_device)
                    
                    delimiter()
                    delimiter()
                    print(f'Starting training for BS: {batch_size} KS: {kernel_size} SS: {sent_size} LR: {learning_rate}')
                    delimiter()
                    
                    models_identifier = create_new_model_input(trait, fold, learning_rate, batch_size, kernel_size, sent_size)
                    
                    model = ConvoCarpet(kernels_count=kernel_size, sentences_count=sent_size)
                    if use_GPU:
                          model.to(cuda_device)

                    optimizer = to.optim.Adam(model.parameters(), learning_rate, amsgrad = True)

                    train_model(train_loader, val_loader, models_identifier, model, loss, optimizer)
                    results.to_csv(RESULTS)
    

def train_model(train_loader, val_loader, models_identifier, model, loss, optimizer):
    best_f1_val = 0
    last_f1 = 0
    counter = 0
    
    for epoch in range(0, n_epochs):
        epoch_start = time.time()
        model.train()
        total_loss = 0
        n_batch = 0
        train_loader, train_loader_backup = tee(train_loader)
        for inputs, labels in train_loader_backup:

            n_batch += 1
            #print(f"Current batch is {n_batch}")

            model.zero_grad()

            logits = model(inputs.to(device=cuda_device, dtype=to.float))
            minibatch_loss = loss(logits, labels)
            total_loss += minibatch_loss
            minibatch_loss.backward()
            optimizer.step()
            if(n_batch % print_status_batch == 0):
                
                delimiter()
                print(f"Train - Epoch {epoch} - batch {n_batch}, batch loss is {minibatch_loss:.6f}, total loss is {total_loss:.6f}")
                delimiter()

                
        delimiter()
        val_loader, val_loader_backup = tee(val_loader)
        val_loss, val_logits, true_val = apply_model(model, loss, val_loader_backup)
        val_f1_score, val_precision_0, val_precision_1, val_precision_m, val_recall_0, val_recall_1, val_recall_m = calcMeasures(val_logits, true_val, 0.5)
        
        
        print(f'Epoch {epoch} end: {time.time()-epoch_start}')
        print(f'Validation loss: {val_loss:.7f} - F1 score: {val_f1_score:.7f}')
        print(f'0 class -> precision: {val_precision_0:.7f} - recall: {val_recall_0:.7f}')
        print(f'1 class -> precision: {val_precision_1:.7f} - recall: {val_recall_1:.7f}')
        print(f'precision: {val_precision_m:.7f} - recall: {val_recall_m:.7f} - MACRO')
        delimiter()
        
        if (abs(val_f1_score - last_f1) <= RELEVANT_DIFFERENCE):
            counter += 1
        else:
            counter = 0
        
        last_f1 = val_f1_score
        
        if(counter >= max_constant_f1):
            return
        
        if (val_f1_score > best_f1_val):
            update_models_val_results(models_identifier, val_f1_score, val_precision_0, val_precision_1, val_precision_m, val_recall_0, val_precision_1, val_recall_m, epoch)
            best_f1_val = val_f1_score
        # save a checkpoint
            best_checkpoint_filename = get_checkpoints_name(models_identifier)
            to.save({
              "models_identifier" : models_identifier, 
              "model_state_dict" : model.state_dict(),
              "optimizer_state_dict" : optimizer.state_dict()
            }, best_checkpoint_filename)
    

In [27]:
for target in mbti_labels:
    print(f"Creating model for target: {target}")
    for fold in range(0, no_of_folds):
        
        print(f'Processing fold {fold}...')
        
        train_input_indices, train_output, val_input_indices, val_output, test_input_indices, test_output = data_preparation(target, fold)
        
        optimize_models_params(target, fold, train_input_indices, train_output, val_input_indices, val_output)
        
        
        best_models = get_best_models_data(target, fold)
        #Recreate model
        models_identifier = best_models['hash_id']
        model = ConvoCarpet(kernels_count=best_models['kernels_count'] , sentences_count=best_models['sentences_count'])
        if use_GPU:
              model.to(cuda_device)

        checkpoint = to.load(get_checkpoints_name(models_identifier))
        model.load_state_dict(checkpoint["model_state_dict"])
        loss = nn.CrossEntropyLoss()

        
        print("."*15)
        print("."*15)
        
        print(f'Apply best model to test for {target} on fold {fold}.')
        test_loader = create_minibatches(test_input_indices, test_output, 1, cuda_device)
        test_loss, test_logits, test_true = apply_model(model, loss, test_loader)
        test_f1_score, test_precision_0, test_precision_1, test_precision_m, test_recall_0, test_recall_1, test_recall_m = calcMeasures(test_logits, test_true, 0.5)
        print(f'Test loss: {test_loss:.5f} - F1 score: {test_f1_score:.7f} ')
        print(f'0 class -> precision: {test_precision_0:.7f} - recall: {test_recall_0:.7f}')
        print(f'1 class -> precision: {test_precision_1:.7f} - recall: {test_recall_1:.7f}')
        print(f'precision: {test_precision_m:.7f} - recall: {test_recall_m:.7f} - MACRO')
              
        update_models_test_results(models_identifier, test_f1_score, test_precision_0, test_precision_1, test_precision_m, test_recall_0, test_recall_1, test_recall_m)
        with open(get_predictions_file_name(models_identifier, target, fold), 'w+') as f:
            for pre_l, true_l in zip(test_logits, test_true):
                f.write(f'{pre_l.cpu()[0][0]}, {pre_l.cpu()[0][1]}, {true_l.cpu().tolist()[0]}\n')
        results.to_csv(RESULTS)
        
        ##Save best models data
        
        print(f"+++ Finished with training and testing model for {target} on fold {fold}. +++")
        
        print("."*15)
        print("."*15)

Creating model for target: introverted
Processing fold 0...
Start training ...
-----------------------
-----------------------
Starting training for BS: 1 KS: 4 SS: 2 LR: 0.001
-----------------------
-----------------------
Train - Epoch 0 - batch 1000, batch loss is 0.313275, total loss is 315.320404
-----------------------
-----------------------
Train - Epoch 0 - batch 2000, batch loss is 0.313270, total loss is 628.598633
-----------------------
-----------------------
Train - Epoch 0 - batch 3000, batch loss is 0.313265, total loss is 941.888611
-----------------------
-----------------------
Train - Epoch 0 - batch 4000, batch loss is 0.313263, total loss is 1255.135742
-----------------------
-----------------------
Train - Epoch 0 - batch 5000, batch loss is 1.313251, total loss is 2135.367676
-----------------------
-----------------------
Train - Epoch 0 - batch 6000, batch loss is 0.313284, total loss is 3334.228516
-----------------------
-----------------------
Train - Ep

/home/mbosnjak/.conda/envs/pero/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/mbosnjak/.conda/envs/pero/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/mbosnjak/.conda/envs/pero/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


Epoch 0 end: 26.23428225517273
Validation loss: 742.9553223 - F1 score: 0.4406646
0 class -> precision: 0.0000000 - recall: 0.0000000
1 class -> precision: 0.7878359 - recall: 1.0000000
precision: 0.3939180 - recall: 0.5000000 - MACRO
-----------------------
-----------------------
Train - Epoch 1 - batch 1000, batch loss is 1.313231, total loss is 1313.236084
-----------------------
-----------------------
Train - Epoch 1 - batch 2000, batch loss is 1.313037, total loss is 2626.452881
-----------------------
-----------------------
Train - Epoch 1 - batch 3000, batch loss is 0.313349, total loss is 3438.257324
-----------------------
-----------------------
Train - Epoch 1 - batch 4000, batch loss is 0.313271, total loss is 3751.576660
-----------------------
-----------------------
Train - Epoch 1 - batch 5000, batch loss is 1.313069, total loss is 4631.661621
-----------------------
-----------------------
Train - Epoch 1 - batch 6000, batch loss is 0.313283, total loss is 5156.6679

-----------------------
Train - Epoch 2 - batch 1000, batch loss is 0.313262, total loss is 313.262756
-----------------------
-----------------------
Train - Epoch 2 - batch 2000, batch loss is 0.313262, total loss is 626.514526
-----------------------
-----------------------
Train - Epoch 2 - batch 3000, batch loss is 0.313262, total loss is 939.746948
-----------------------
-----------------------
Train - Epoch 2 - batch 4000, batch loss is 0.313262, total loss is 1252.979370
-----------------------
-----------------------
Train - Epoch 2 - batch 5000, batch loss is 1.313262, total loss is 2133.211914
-----------------------
-----------------------
Train - Epoch 2 - batch 6000, batch loss is 1.313262, total loss is 3446.444336
-----------------------
-----------------------
Train - Epoch 2 - batch 7000, batch loss is 1.313262, total loss is 4759.800293
-----------------------
-----------------------
Train - Epoch 2 - batch 8000, batch loss is 1.313262, total loss is 6073.276855
---

-----------------------
Train - Epoch 3 - batch 3000, batch loss is 0.416072, total loss is 2636.779297
-----------------------
-----------------------
Train - Epoch 3 - batch 4000, batch loss is 0.407938, total loss is 3048.677734
-----------------------
-----------------------
Train - Epoch 3 - batch 5000, batch loss is 0.315648, total loss is 3595.943115
-----------------------
-----------------------
Train - Epoch 3 - batch 6000, batch loss is 0.313866, total loss is 3910.662598
-----------------------
-----------------------
Train - Epoch 3 - batch 7000, batch loss is 0.313548, total loss is 4224.463379
-----------------------
-----------------------
Train - Epoch 3 - batch 8000, batch loss is 0.313491, total loss is 4538.075684
-----------------------
-----------------------
Epoch 3 end: 18.02345371246338
Validation loss: 743.0032959 - F1 score: 0.4406646
0 class -> precision: 0.0000000 - recall: 0.0000000
1 class -> precision: 0.7878359 - recall: 1.0000000
precision: 0.3939180 -

...

-----------------------
Train - Epoch 3 - batch 1000, batch loss is 0.313262, total loss is 313.262756
-----------------------
-----------------------
Train - Epoch 3 - batch 2000, batch loss is 1.313262, total loss is 1518.252319
-----------------------
-----------------------
Epoch 3 end: 7.539532899856567
Validation loss: 389.8950195 - F1 score: 0.1750292
0 class -> precision: 0.2121641 - recall: 1.0000000
1 class -> precision: 0.0000000 - recall: 0.0000000
precision: 0.1060820 - recall: 0.5000000 - MACRO
-----------------------
-----------------------
Train - Epoch 4 - batch 1000, batch loss is 0.313262, total loss is 313.262756
-----------------------
-----------------------
Train - Epoch 4 - batch 2000, batch loss is 1.313262, total loss is 1518.252319
-----------------------
-----------------------
Epoch 4 end: 7.4517738819122314
Validation loss: 389.8950195 - F1 score: 0.1750292
0 class -> precision: 0.2121641 - recall: 1.0000000
1 class -> precision: 0.0000000 - recall: 0.0000

-----------------------
Train - Epoch 3 - batch 1000, batch loss is 0.585359, total loss is 670.458801
-----------------------
-----------------------
Train - Epoch 3 - batch 2000, batch loss is 0.648652, total loss is 1389.542725
-----------------------
-----------------------
Epoch 3 end: 7.535017013549805
Validation loss: 226.1764221 - F1 score: 0.4404432
0 class -> precision: 0.0000000 - recall: 0.0000000
1 class -> precision: 0.7876858 - recall: 0.9991023
precision: 0.3938429 - recall: 0.4995512 - MACRO
-----------------------
-----------------------
Train - Epoch 4 - batch 1000, batch loss is 0.593930, total loss is 681.672180
-----------------------
-----------------------
Train - Epoch 4 - batch 2000, batch loss is 0.640300, total loss is 1391.859619
-----------------------
-----------------------
Epoch 4 end: 7.185239315032959
Validation loss: 224.8370209 - F1 score: 0.4406646
0 class -> precision: 0.0000000 - recall: 0.0000000
1 class -> precision: 0.7878359 - recall: 1.00000

-----------------------
Train - Epoch 9 - batch 2000, batch loss is 0.726937, total loss is 1381.140747
-----------------------
-----------------------
Epoch 9 end: 7.020796060562134
Validation loss: 248.7260284 - F1 score: 0.3778316
0 class -> precision: 0.2212560 - recall: 0.7633333
1 class -> precision: 0.8126649 - recall: 0.2764811
precision: 0.5169605 - recall: 0.5199072 - MACRO
-----------------------
-----------------------
Train - Epoch 10 - batch 1000, batch loss is 0.685039, total loss is 666.243469
-----------------------
-----------------------
Train - Epoch 10 - batch 2000, batch loss is 0.725652, total loss is 1381.163574
-----------------------
-----------------------
Epoch 10 end: 7.152132987976074
Validation loss: 248.4920044 - F1 score: 0.3878409
0 class -> precision: 0.2225519 - recall: 0.7500000
1 class -> precision: 0.8138958 - recall: 0.2944345
precision: 0.5182239 - recall: 0.5222172 - MACRO
-----------------------
-----------------------
Train - Epoch 11 - batch

-----------------------
Epoch 24 end: 7.463612079620361
Validation loss: 246.0101318 - F1 score: 0.4435238
0 class -> precision: 0.2168367 - recall: 0.5666667
1 class -> precision: 0.7936508 - recall: 0.4488330
precision: 0.5052438 - recall: 0.5077499 - MACRO
-----------------------
-----------------------
Train - Epoch 25 - batch 1000, batch loss is 0.696017, total loss is 676.742371
-----------------------
-----------------------
Train - Epoch 25 - batch 2000, batch loss is 0.710980, total loss is 1380.878662
-----------------------
-----------------------
Epoch 25 end: 7.448807239532471
Validation loss: 245.8781433 - F1 score: 0.4484989
0 class -> precision: 0.2193548 - recall: 0.5666667
1 class -> precision: 0.7965571 - recall: 0.4569120
precision: 0.5079560 - recall: 0.5117893 - MACRO
-----------------------
-----------------------
Train - Epoch 26 - batch 1000, batch loss is 0.696502, total loss is 677.228882
-----------------------
-----------------------
Train - Epoch 26 - batc

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

